# Extrahieren der Textstufen

In [1]:
import lxml
from lxml import etree
import pandas as pd

In [2]:
poem_id = "aptrm"

In [3]:
tree = etree.parse(f"C:/Users/jO/Desktop/DH_MA_Arbeit/coding/{poem_id}/modelliert_output_text+pos_{poem_id}_trf.xml")

In [4]:
ns = {"t":'http://www.tei-c.org/ns/1.0'}
xml = "{http://www.w3.org/XML/1998/namespace}"

## Funktion:

In [5]:

processed_divs = set() # zur Nachverfolgung, welche Divs bereits verarbeitet wurden
processed_texts = set() # zur Nachverfolgung, welche Fassungen bereits verarbeitet wurden

#Funktion, die den extrahierten xpath-text weiterverarbeitet, die Tag-Hierarchie erstellt und den Text im XML-Dokument speichert
def addtext(text, div, textname): 
    if textname not in processed_texts:    
        text = "".join(text)
        text = text.replace("\n", "").replace("\r", "")
        text = " ".join(text.strip().split())
        print(f"\n Neuer extrahierter Text:\n > Name: {textname}: \n > Text: ---")
        print(f"\n Der Text von '{textname}' wird in xml geladen ...")
        linetag = etree.Element("l")
        print("1. linetag wurde erstellt")
        linetag.text = text # den extrahierten Text als Inhalt eines Linetags definieren
        print("2. text wurde dem linetag hinzugefügt")
        linegrouptag = etree.Element("lg")  
        print("3. linegrouptag wurde erstellt")
        linegrouptag.append(linetag) # Linetag dem Linegrouptag hinzufügen
        print("4. linetag wurde dem linegrouptag hinzugefügt")
        divtag = etree.Element("div") # div erstellen
        divtag.set("type", f"{textname}")  # Attribut für div mit dem Attributwert der spezifischen Fassung festlegen    
        print("5. divtag wurde erstellt")
        divtag.append(linegrouptag) # Linegrouptag dem divtag hinzufügen
        print("6. linegrouptag wurde zu divtag hinzugefügt")
        #div.append(divtag) # in das Ursprungs-div das extrahierte Textstufen-div hinzufügen -> macht keinen Sinn!!
        #print("7. divtag wurde dem div hinzugefügt")  
        div.addnext(divtag) # nach dem Ursprungs-div das extrahierte Textstufen-div hinzufügen
        print(f" ... und '{textname}' wurde zum xml hinzugefügt!\n")
        processed_texts.add(textname)
        if textname in processed_texts: 
            print(f"verarbeitete texte sind: {processed_texts}")
            return
    else:
        print(f"{textname} wurde schon verarbeitet!")
        return    
        
def processalldivs(poem_id):
    divs = tree.xpath(f"//t:div[@type='poem' and @xml:id='{poem_id}']/t:div[@type='witness']", namespaces=ns)
    divlen = len(divs)
    counter = 0 # Counter, um die Anzahl verarbeiteter divs im Auge zu behalten
    print(f"Es sind {len(divs)} divs im XML-Dokument")
    if counter <= divlen:
        for div in divs: # Attribute des divs abrufen, um eine ID == divname zu kreieren
            divtype = div.attrib.get('type')
            divid = div.attrib.get(f'{xml}id')
            divname = f"{divtype}_{divid}"
            print(f"! Für folgendes <div> werden alle @change-Versionen extrahiert: '{divname}' !")
            if divname not in processed_divs:
                #"original"-Version herausnehmen (=heißt: den Text in <l>, <del> und <orig> beachten):
                textname = f"{divname}_original"
                text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:del/text() | 
                //t:div[@type='{divtype}' and @xml:id='{divid}']//t:orig/text()""", namespaces=ns)
                addtext(text, div, textname) #Funktion zur Verarbeitung und Speicherung des extrahierten Textes
            
                allattributevalues = set() #set für alle einzigartigen @change-Attributevalues pro div   
                
                # der Text mit den Revisions-Tags befindet sich jeweils im Linetag <l>:
                lines = tree.xpath(f"//t:div[@type='poem' and @xml:id='{poem_id}']//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/descendant-or-self::*", namespaces=ns)
                
                for line in lines:
                    attribute_value = line.attrib.get('change') #unterschiedliche values von @change in allen tags innerhalb einer line abrufen
                    if attribute_value: 
                        if attribute_value not in allattributevalues:
                            allattributevalues.add(attribute_value) # unterschiedliche values von @change in das set hinzufügen
                print(f"\n Es gibt folgende Revisionen im Text '{divname}': {allattributevalues}\n") # alle gefundenen @change-values printen 
                
                for attributevalue in allattributevalues: # pro @change-value, das im set gespeichert wurde, wird der spezifische Revisionstext abgerufen
                    textname = f"{divname}_{attributevalue}"
                    # pro Change Text beachten in <l>, <add>, <reg> aber nicht in <del> & <orig> (wenn reg dasselbe value hat wie <orig>)
                    text = tree.xpath(f"""//t:div[@type='{divtype}' and @xml:id='{divid}']//t:l/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:add[@change='{attributevalue}']/text() | 
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:del[not(@change='{attributevalue}')]/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:l//t:choice/t:reg[@change='{attributevalue}']/text() |
                    //t:div[@type='{divtype}' and @xml:id='{divid}']//t:choice[(t:reg[not(@change='{attributevalue}')])]/t:orig/text()""", 
                                      namespaces=ns)
                    addtext(text, div, textname) #Funktion zur Verarbeitung und Speicherung des Textes
                processed_divs.add(divname)
                counter += 1
                print(f"\n!!! {counter} div/s wurden verarbeitet!\n")
            else:
                print("All done")
                return


In [6]:
processalldivs(poem_id)

tree.write(f"extrahiert_{poem_id}.xml", encoding="utf-8", xml_declaration=True)


Es sind 4 divs im XML-Dokument
! Für folgendes <div> werden alle @change-Versionen extrahiert: 'witness_nr.1' !

 Neuer extrahierter Text:
 > Name: witness_nr.1_original: 
 > Text: ---

 Der Text von 'witness_nr.1_original' wird in xml geladen ...
1. linetag wurde erstellt
2. text wurde dem linetag hinzugefügt
3. linegrouptag wurde erstellt
4. linetag wurde dem linegrouptag hinzugefügt
5. divtag wurde erstellt
6. linegrouptag wurde zu divtag hinzugefügt
 ... und 'witness_nr.1_original' wurde zum xml hinzugefügt!

verarbeitete texte sind: {'witness_nr.1_original'}

 Es gibt folgende Revisionen im Text 'witness_nr.1': {'#blue_pen'}


 Neuer extrahierter Text:
 > Name: witness_nr.1_#blue_pen: 
 > Text: ---

 Der Text von 'witness_nr.1_#blue_pen' wird in xml geladen ...
1. linetag wurde erstellt
2. text wurde dem linetag hinzugefügt
3. linegrouptag wurde erstellt
4. linetag wurde dem linegrouptag hinzugefügt
5. divtag wurde erstellt
6. linegrouptag wurde zu divtag hinzugefügt
 ... und 'wit